# Evaluation of UNET performance upon common distortions

The aim of this notebook is to provide some informations about how a UNET initially trained to segment cells is perturbated by some common distortions applied on the input images. The tested distortions applied on the inputs are an added 2D gaussian, a gaussian noise as well as a rescaling of the input images. To evaluate the performance of the UNET, several plots are generated: the accuracy, the Jaccard index and the number of detected cells in function of the degree of degradation that depends on the parameters of the distortions.

### $\bullet$ Importing libraries and utilitary functions

In [ ]:
# import python libraries
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow

# import distortion filters and utilitary functions
from distortions import add_gaussian, zoom_image, zoom_image_to_meet_shape, add_gaussian_noise
from utils import *
from plots import show_image_mask, show_image_pred, plot_all
from image_processing_methods import dog

### $\bullet$ Setting size of the input image and different paths

In [ ]:
# Shape of the inputs of the deep network.
images_shape = (256, 256, 1)

# Variables defining the path to the dataset.
test_input_path = '../Dataset/test/input/'
test_output_path = '../Dataset/test/output/'

# Variable defining where model is stored.
model_save_path = "../Model/"

### $\bullet$ Function to display the input images and predictions upon distorsion as well as a summary of values that shows the quality of the predicted images compared to the ground truth

In [ ]:
def evaluation_summary(result, parameter_name, parameter_val, image, mask, distorted_image, model):
    """Eval.
    
    Args:
        images::[np.array]
            The output measures associated to the input measures tX.
        masks::[np.array]
            The input measures.
    
    """
    print("{:<45}: {}".format(parameter_name.upper()          , parameter_val))
    print("{:<45}: {}".format("Accuracy"                      , result["accuracy"]))
    print("{:<45}: {}".format("Jaccard score"                 , result["jaccard"]))
    print("{:<45}: {}".format("Precision"                     , result["precision"]))
    print("{:<45}: {}".format("Recall"                        , result["recall"]))
    print("{:<45}: {}".format("Number of cells in predictions", result["number_cells_predictions"]))
    print("{:<45}: {}".format("Number of cells in masks"      , result["number_cells_masks"]))
    show_image_mask(image, mask)
    show_image_pred(distorted_image, model)
    
    
def evaluate_model(images, masks, model, number_cells_masks=None):
    """Eval.
    
    Args:
        images::[np.array]
            The output measures associated to the input measures tX.
        masks::[np.array]
            The input measures.
    Returns:
        accuracy::[float]
            An array comprised of n_folds equally distributed arrays of y.
        jaccard_score::[float]
            An array comprised of n_folds equally distributed arrays of tX.
        number_cells_predictions::[int]
        number_cells_masks::[int]
    
    """
    predictions = get_binary_predictions(images, model)
    accuracy    = np.mean(predictions == masks)
    jaccard     = compute_jaccard_score(predictions, masks)
    precision, recall = compute_precision_recall(predictions, masks)

    number_cells_predictions = get_number_cells(predictions)
    if number_cells_masks is None:
        number_cells_masks = get_number_cells(masks)
    
    return {"accuracy": accuracy, "jaccard": jaccard, "precision": precision, 
            "recall": recall, "number_cells_predictions": number_cells_predictions,
            "number_cells_masks": number_cells_masks}


def apply_distortion_to_all(function, images, params_for_images={}):
    distorted_images = []
    for image in images:
        distorted_images.append(function(image, **params_for_images))
    
    return np.array(distorted_images)

### $\bullet$ Retrieve trained model

In [ ]:
model = tensorflow.keras.models.load_model(model_save_path)

### $\bullet$ Get input images with the wanted shape from dataset

In [ ]:
test_images, test_masks = get_dataset_from_folders(test_input_path, test_output_path, images_shape)

print(f'Test set contains {len(test_images)} images of shape {test_images[0].shape}.')

number_cells_masks = get_number_cells(test_masks)
print(f"{number_cells_masks} cells were counted in total over all masks.")

In [ ]:
from tqdm import tqdm

In [ ]:
amplitudes = np.linspace(0, 4000, 10)

all_images = np.zeros((510*10, 256, 256, 1))

for i,amplitude in enumerate(amplitudes):
    all_images[510*i:510*(1+i), :, :, :] = apply_distortion_to_all(add_gaussian, test_images, {"amplitude": amplitude})
    
all_masks = np.repeat(test_masks, 10, axis=0)

In [ ]:
sigmas = np.linspace(0.5,0.9,9) 

dist_sigma = []
for sigma in sigmas:
    dist = []
    indeces = np.random.choice(all_images.shape[0], 1000)
    for i in tqdm(indeces):
        dog_added_gaussian_img = dog(all_images[i], sigma)
        dist.append(cv2.norm(all_masks[i]-dog_added_gaussian_img, cv2.NORM_L1))
    
    dist_sigma.append(np.mean(dist))


In [ ]:
sigma = 0.65
indeces = np.random.choice(all_images.shape[0], 100)
for i in tqdm([-1]):
    dog_added_gaussian_img = dog(all_images[i], sigma)
    #m = np.median(dog_added_gaussian_img)
    #pred = dog_added_gaussian_img >= m
    
    plt.figure()
    plt.imshow(dog_added_gaussian_img, 'gray', vmin=0, vmax=1)
    plt.show()
    
    dog_added_gaussian_img = dog_added_gaussian_img.reshape((1, 256, 256, 1))
    
    result = evaluate_model(dog_added_gaussian_img*256, test_masks[-1], model)
    
    evaluation_summary(result, "Amplitude before DOG", 4000, test_images[-1], test_masks[-1], dog_added_gaussian_img.reshape((256, 256, 1))*256, model)

In [ ]:
plt.plot(sigmas, dist_sigma)

In [ ]:
best_dists = []
best_sigmas = []
for img, mask in zip(test_images, test_masks):
    added_gaussian_img = add_gaussian(img, 1000)

    sigmas = np.linspace(0.1,1.9,20)
    distances = []
    for sigma in sigmas:
        fig, axes = plt.subplots(1, 3)
        ax1, ax2, ax3 = axes

        dog_added_gaussian_img = dog(added_gaussian_img, sigma)
        ax1.imshow(img, 'gray')
        ax1.axis('off')
        ax2.imshow(added_gaussian_img, 'gray')
        ax2.axis('off')
        ax3.imshow(dog_added_gaussian_img, 'gray')
        ax3.axis('off')
        plt.show()
        
        distances.append(np.sum(np.abs(mask-dog_added_gaussian_img)))
        print("sigma: ", sigma)
        print("distance: ", np.sum(np.abs(mask-dog_added_gaussian_img)))
        
    best_dist = np.min(distances)
    best_sigma = sigmas[np.argmin(distances)]
    best_dists.append(best_dist)
    best_sigmas.append(best_sigma)
    plt.plot(sigmas, distances)
    plt.scatter(best_sigma, best_dist)
    


In [ ]:
plt.plot(sigmas, distances)

In [ ]:
from noise_removal import de_noise

for img, mask in list(zip(test_images, test_masks))[:1]:
    
    sigmas = np.linspace(1,100,40)
    distances = []
    for sigma in sigmas:
        added_noise_img = add_gaussian_noise(img, 0, sigma)
        
        de_noised_img = de_noise(np.array(added_noise_img, dtype=np.uint8))

        fig, axes = plt.subplots(1, 3)
        ax1, ax2, ax3 = axes

        ax1.imshow(img, 'gray')
        ax1.axis('off')
        ax2.imshow(added_noise_img, 'gray')
        ax2.axis('off')
        ax3.imshow(de_noised_img, 'gray')
        ax3.axis('off')
        plt.show()
        print("sigma: ", sigma)
        distance = np.sum(np.abs(img - de_noised_img))
        print("distance: ", distance)
        distances.append(distance)
        

    best_dist = np.min(distances)
    best_sigma = sigmas[np.argmin(distances)]
    best_dists.append(best_dist)
    best_sigmas.append(best_sigma)
    plt.plot(sigmas, distances)
    plt.scatter(best_sigma, best_dist)

In [ ]:
sigma = 40
indeces = np.random.choice(all_images.shape[0], 100)
for i in tqdm([-1]):
    added_noise_img = add_gaussian_noise(test_images[0], 0, sigma)
    de_noised_img = de_noise(np.array(added_noise_img, dtype=np.uint8))
        
    de_noised_img = de_noised_img.reshape((1, 256, 256, 1))
    
    result = evaluate_model(de_noised_img*256, test_masks[0], model)
    
    evaluation_summary(result, "Model after denoiser", 0, test_images[0], test_masks[0], de_noised_img.reshape((256, 256, 1))*256, model)